In [5]:
import numpy as np
import math

import matplotlib.pyplot as plt
import cartopy.crs as crs
import cartopy.feature as cfeature
import matplotlib.colors as colors
from matplotlib.cm import get_cmap
from matplotlib import ticker
import matplotlib.gridspec as gridspec

from cartopy import config
import cartopy.crs as ccrs
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter
from cartopy.util import add_cyclic_point

from IPython.display import Image

from tqdm import tqdm
import os 

In [6]:
directory = '/n/home12/hongwei/HONGWEI/lagranto.ecmwf/Simulation/run_10yr_2000'

filename01 = ['/traj.20000101','/traj.20000102','/traj.20000103', \
              '/traj.20000104','/traj.20000105','/traj.20000106', \
              '/traj.20000107','/traj.20000108','/traj.20000109']

filename04 = ['/traj.20000401','/traj.20000402','/traj.20000403', \
              '/traj.20000404','/traj.20000405','/traj.20000406', \
              '/traj.20000407','/traj.20000408','/traj.20000409']

filename07 = ['/traj.20000701','/traj.20000702','/traj.20000703', \
              '/traj.20000704','/traj.20000705','/traj.20000706', \
              '/traj.20000707','/traj.20000708','/traj.20000709']

filename10 = ['/traj.20001001','/traj.20001002','/traj.20001003', \
              '/traj.20001004','/traj.20001005','/traj.20001006', \
              '/traj.20001007','/traj.20001008','/traj.20001009']

In [7]:
N_head = 5 # first 5 lines are head lines, not include data
N_column = 4

Nx = 36
Ny = 21
Nz = 5
N_parcel = 3780

if Nx*Ny*Nz!=N_parcel: print('ERROR: parcel number is wrong!')
    
N_inject = 9 # how many injecting times



In [8]:
# loop for all traj files for month 01

file1 = open(directory+filename01[0], 'r')
Lines = file1.readlines()
Nt = int( (len(Lines)-4)/N_parcel - 1 ) # 2928

lifetime_01 = np.zeros((Nz,Ny,Nx, N_inject))

for i_inject in tqdm(range(N_inject)):
    
    # (1) read original data from traj files
    data = np.zeros((N_parcel, Nt, N_column))

    file1 = open(directory+filename01[i_inject], 'r')
#     print('Begin: '+filename01[i_inject])
    Lines = file1.readlines()

    count = 0
    # Strips the newline character
    for line in Lines:
        count += 1
    
        if count>=5:
            i = count-5
            i_parcel = math.floor( i / (Nt+1) ) # Nt time lines plue 1 empty line
            i_t = i%(Nt+1)
        
            if i_t!=0:
                a = line.split()

                if i_t==1:
                    if float(a[0])!=0.0: print('ERROR: first time is not 0 !!!')
                
                data[i_parcel,i_t-1,1] = float(a[1]) # lon
                data[i_parcel,i_t-1,2] = float(a[2]) # lat
                data[i_parcel,i_t-1,3] = float(a[3]) # lev
                
                
    # (2) re-shape data(N_parcel, Nt, N_column) to data2(Nz,Ny,Nx,Nt,N_column)
    data2 = np.zeros((Nz,Ny,Nx,Nt,N_column))

    for iy in range(Ny):
        for iz in range(Nz):
            for ix in range(Nx):
                i_parcel = iy*(Nz*Nx) + iz*Nx + ix
                data2[iz,iy,ix,:,:] = data[i_parcel,:,:]
                
                
    # (3) calculate injected tracer lifetime in the stratosphere
    P_cont = 200 # [hPa]
    lifetime = np.zeros((Nz,Ny,Nx))

    for iy in range(Ny):
        for iz in range(Nz):
            for ix in range(Nx):
                Num = 0
                Num_N = 0
                Num_S = 0
                for i in data2[iz,iy,ix,:,3]:
                    Num = Num+1
                    if i>=P_cont: 
                        lifetime[iz,iy,ix] = Num*3/24 # hour to day
                        break
                
                
    # (4) create 2-D LAT and LON
    LON = np.zeros((Ny,Nx))
    LAT = np.zeros((Ny,Nx))

    for iy in range(Ny):
        for ix in range(Nx):
            LON[iy,ix] = data2[0,iy,ix,0,1]
            LAT[iy,ix] = data2[0,iy,ix,0,2]

            
    # (5) Sort lat dimension from min to max [-20:20]
    index = np.argsort(LAT[:,0])

    LAT[:,:] = LAT[index,:]
    LON[:,:] = LON[index,:]
    lifetime[:,:,:] = lifetime[:,index,:]
    data2[:,:,:,:,:] = data2[:,index,:,:,:]
              
    # (6) Finally, save lifetime into  lifetime_01[Nz,Ny,Nx,N_inject]
    lifetime_01[:,:,:,i_inject] = lifetime[:,:,:]
        
    file1.close()

  0%|          | 0/9 [00:00<?, ?it/s]

Begin: /traj.20000101


 11%|█         | 1/9 [04:41<37:32, 281.52s/it]

Begin: /traj.20000102


 22%|██▏       | 2/9 [09:27<33:09, 284.19s/it]

...

 78%|███████▊  | 7/9 [33:20<09:32, 286.17s/it]

Begin: /traj.20000108


 89%|████████▉ | 8/9 [38:07<04:46, 286.74s/it]

Begin: /traj.20000109


100%|██████████| 9/9 [42:58<00:00, 286.49s/it]


In [13]:
# save lifetime_01[Nz,Ny,Nx,N_inject] to Lifetime01.txt

with open('Lifetime01.txt', 'w') as f:
    for i_inject in range(N_inject):
        for ix in range(Nx):
            for iy in range(Ny):
                for iz in range(Nz):
                    f.write(str(lifetime_01[iz,iy,ix,i_inject]))
                    f.write('\n')    
f.close()

In [10]:
# loop for all traj files for month 04

file1 = open(directory+filename04[0], 'r')
Lines = file1.readlines()
Nt = int( (len(Lines)-4)/N_parcel - 1 ) # 2928

lifetime_04 = np.zeros((Nz,Ny,Nx, N_inject))

for i_inject in tqdm(range(N_inject)):
    
    # (1) read original data from traj files
    data = np.zeros((N_parcel, Nt, N_column))

    file1 = open(directory+filename04[i_inject], 'r')
#     print('Begin: '+filename04[i_inject])
    Lines = file1.readlines()

    count = 0
    # Strips the newline character
    for line in Lines:
        count += 1
    
        if count>=5:
            i = count-5
            i_parcel = math.floor( i / (Nt+1) ) # Nt time lines plue 1 empty line
            i_t = i%(Nt+1)
        
            if i_t!=0:
                a = line.split()

                if i_t==1:
                    if float(a[0])!=0.0: print('ERROR: first time is not 0 !!!')
                
                data[i_parcel,i_t-1,1] = float(a[1]) # lon
                data[i_parcel,i_t-1,2] = float(a[2]) # lat
                data[i_parcel,i_t-1,3] = float(a[3]) # lev
                
                
    # (2) re-shape data(N_parcel, Nt, N_column) to data2(Nz,Ny,Nx,Nt,N_column)
    data2 = np.zeros((Nz,Ny,Nx,Nt,N_column))

    for iy in range(Ny):
        for iz in range(Nz):
            for ix in range(Nx):
                i_parcel = iy*(Nz*Nx) + iz*Nx + ix
                data2[iz,iy,ix,:,:] = data[i_parcel,:,:]
                
                
    # (3) calculate injected tracer lifetime in the stratosphere
    P_cont = 200 # [hPa]
    lifetime = np.zeros((Nz,Ny,Nx))

    for iy in range(Ny):
        for iz in range(Nz):
            for ix in range(Nx):
                Num = 0
                for i in data2[iz,iy,ix,:,3]:
                    Num = Num+1
                    if i>=P_cont: 
                        lifetime[iz,iy,ix] = Num*3/24 # hour to day
                        break
                
                
    # (4) create 2-D LAT and LON
    LON = np.zeros((Ny,Nx))
    LAT = np.zeros((Ny,Nx))

    for iy in range(Ny):
        for ix in range(Nx):
            LON[iy,ix] = data2[0,iy,ix,0,1]
            LAT[iy,ix] = data2[0,iy,ix,0,2]

            
    # (5) Sort lat dimension from min to max [-20:20]
    index = np.argsort(LAT[:,0])

    LAT[:,:] = LAT[index,:]
    LON[:,:] = LON[index,:]
    lifetime[:,:,:] = lifetime[:,index,:]
    data2[:,:,:,:,:] = data2[:,index,:,:,:]
              
    # (6) Finally, save lifetime into  lifetime_04[Nz,Ny,Nx,N_inject]
    lifetime_04[:,:,:,i_inject] = lifetime[:,:,:]
        
    file1.close()

100%|██████████| 9/9 [39:37<00:00, 264.19s/it]


In [11]:
# save lifetime_04[Nz,Ny,Nx,N_inject] to Lifetime04.txt

with open('Lifetime04.txt', 'w') as f:
    for i_inject in range(N_inject):
        for ix in range(Nx):
            for iy in range(Ny):
                for iz in range(Nz):
                    f.write(str(lifetime_04[iz,iy,ix,i_inject]))
                    f.write('\n')    
f.close()

In [12]:
# loop for all traj files for month 07

file1 = open(directory+filename07[0], 'r')
Lines = file1.readlines()
Nt = int( (len(Lines)-4)/N_parcel - 1 ) # 2928

lifetime_07 = np.zeros((Nz,Ny,Nx, N_inject))

for i_inject in tqdm(range(N_inject)):
    
    # (1) read original data from traj files
    data = np.zeros((N_parcel, Nt, N_column))

    file1 = open(directory+filename07[i_inject], 'r')
#     print('Begin: '+filename07[i_inject])
    Lines = file1.readlines()

    count = 0
    # Strips the newline character
    for line in Lines:
        count += 1
    
        if count>=5:
            i = count-5
            i_parcel = math.floor( i / (Nt+1) ) # Nt time lines plue 1 empty line
            i_t = i%(Nt+1)
        
            if i_t!=0:
                a = line.split()

                if i_t==1:
                    if float(a[0])!=0.0: print('ERROR: first time is not 0 !!!')
                
                data[i_parcel,i_t-1,1] = float(a[1]) # lon
                data[i_parcel,i_t-1,2] = float(a[2]) # lat
                data[i_parcel,i_t-1,3] = float(a[3]) # lev
                
                
    # (2) re-shape data(N_parcel, Nt, N_column) to data2(Nz,Ny,Nx,Nt,N_column)
    data2 = np.zeros((Nz,Ny,Nx,Nt,N_column))

    for iy in range(Ny):
        for iz in range(Nz):
            for ix in range(Nx):
                i_parcel = iy*(Nz*Nx) + iz*Nx + ix
                data2[iz,iy,ix,:,:] = data[i_parcel,:,:]
                
                
    # (3) calculate injected tracer lifetime in the stratosphere
    P_cont = 200 # [hPa]
    lifetime = np.zeros((Nz,Ny,Nx))

    for iy in range(Ny):
        for iz in range(Nz):
            for ix in range(Nx):
                Num = 0
                for i in data2[iz,iy,ix,:,3]:
                    Num = Num+1
                    if i>=P_cont: 
                        lifetime[iz,iy,ix] = Num*3/24 # hour to day
                        break
                
                
    # (4) create 2-D LAT and LON
    LON = np.zeros((Ny,Nx))
    LAT = np.zeros((Ny,Nx))

    for iy in range(Ny):
        for ix in range(Nx):
            LON[iy,ix] = data2[0,iy,ix,0,1]
            LAT[iy,ix] = data2[0,iy,ix,0,2]

            
    # (5) Sort lat dimension from min to max [-20:20]
    index = np.argsort(LAT[:,0])

    LAT[:,:] = LAT[index,:]
    LON[:,:] = LON[index,:]
    lifetime[:,:,:] = lifetime[:,index,:]
    data2[:,:,:,:,:] = data2[:,index,:,:,:]
              
    # (6) Finally, save lifetime into  lifetime_07[Nz,Ny,Nx,N_inject]
    lifetime_07[:,:,:,i_inject] = lifetime[:,:,:]
        
    file1.close()

100%|██████████| 9/9 [40:51<00:00, 272.42s/it]


In [13]:
# save lifetime_07[Nz,Ny,Nx,N_inject] to Lifetime07.txt

with open('Lifetime07.txt', 'w') as f:
    for i_inject in range(N_inject):
        for ix in range(Nx):
            for iy in range(Ny):
                for iz in range(Nz):
                    f.write(str(lifetime_07[iz,iy,ix,i_inject]))
                    f.write('\n')    
f.close()

In [8]:
# loop for all traj files for month 10

file1 = open(directory+filename10[0], 'r')
Lines = file1.readlines()
Nt = int( (len(Lines)-4)/N_parcel - 1 ) # 2928

lifetime_10 = np.zeros((Nz,Ny,Nx, N_inject))

for i_inject in tqdm(range(N_inject)):
    
    # (1) read original data from traj files
    data = np.zeros((N_parcel, Nt, N_column))

    file1 = open(directory+filename10[i_inject], 'r')
#     print('Begin: '+filename10[i_inject])
    Lines = file1.readlines()

    count = 0
    # Strips the newline character
    for line in Lines:
        count += 1
    
        if count>=5:
            i = count-5
            i_parcel = math.floor( i / (Nt+1) ) # Nt time lines plue 1 empty line
            i_t = i%(Nt+1)
        
            if i_t!=0:
                a = line.split()

                if i_t==1:
                    if float(a[0])!=0.0: print('ERROR: first time is not 0 !!!')
                
                data[i_parcel,i_t-1,1] = float(a[1]) # lon
                data[i_parcel,i_t-1,2] = float(a[2]) # lat
                data[i_parcel,i_t-1,3] = float(a[3]) # lev
                
                
    # (2) re-shape data(N_parcel, Nt, N_column) to data2(Nz,Ny,Nx,Nt,N_column)
    data2 = np.zeros((Nz,Ny,Nx,Nt,N_column))

    for iy in range(Ny):
        for iz in range(Nz):
            for ix in range(Nx):
                i_parcel = iy*(Nz*Nx) + iz*Nx + ix
                data2[iz,iy,ix,:,:] = data[i_parcel,:,:]
                
                
    # (3) calculate injected tracer lifetime in the stratosphere
    P_cont = 200 # [hPa]
    lifetime = np.zeros((Nz,Ny,Nx))

    for iy in range(Ny):
        for iz in range(Nz):
            for ix in range(Nx):
                Num = 0
                for i in data2[iz,iy,ix,:,3]:
                    Num = Num+1
                    if i>=P_cont: 
                        lifetime[iz,iy,ix] = Num*3/24 # hour to day
                        break
                
                
    # (4) create 2-D LAT and LON
    LON = np.zeros((Ny,Nx))
    LAT = np.zeros((Ny,Nx))

    for iy in range(Ny):
        for ix in range(Nx):
            LON[iy,ix] = data2[0,iy,ix,0,1]
            LAT[iy,ix] = data2[0,iy,ix,0,2]

            
    # (5) Sort lat dimension from min to max [-20:20]
    index = np.argsort(LAT[:,0])

    LAT[:,:] = LAT[index,:]
    LON[:,:] = LON[index,:]
    lifetime[:,:,:] = lifetime[:,index,:]
    data2[:,:,:,:,:] = data2[:,index,:,:,:]
              
    # (6) Finally, save lifetime into  lifetime_10[Nz,Ny,Nx,N_inject]
    lifetime_10[:,:,:,i_inject] = lifetime[:,:,:]
        
    file1.close()

100%|██████████| 9/9 [38:01<00:00, 253.55s/it]


In [9]:
# save lifetime_10[Nz,Ny,Nx,N_inject] to Lifetime10.txt

with open('Lifetime10.txt', 'w') as f:
    for i_inject in range(N_inject):
        for ix in range(Nx):
            for iy in range(Ny):
                for iz in range(Nz):
                    f.write(str(lifetime_10[iz,iy,ix,i_inject]))
                    f.write('\n')    
f.close()